# Getting Powerplant info from Openstreet Map 

Useful links: http://wiki.openstreetmap.org/wiki/Overpass_API, https://github.com/mvexel/overpass-api-python-wrapper
Overpass turbo: http://overpass-turbo.eu/

In [1]:
%%time
import overpass

#set timeout higher 
api = overpass.API(timeout=900)
nodes = api.Get('node["power" = "plant"]')
#print(nodes)

Wall time: 11.4 s


In [3]:
%%time
api = overpass.API(timeout=900)
ways = api.Get('way["power" = "plant"]')
#print(ways)

Wall time: 2min 19s


You can see it takes less than three minutes to run. There are many more ways than nodes. A closed way is a polygon and represents the perimeter of something (powerplant in this case).

goal -> find an elegant way to switch the order of the coordinates bc they are appearing in reverse
I would like to reverse the order of the coordinates in this JSON response from (lon,lat) to (lat,lon) so that the 
reverse geocoder will work

In [4]:
import json
import requests
import numpy as np

#dict.iteritems() changed to dict.items() in python 3

def flip_geojson_coordinates(geo):
    if isinstance(geo, dict):
        for k, v in geo.items():
            if k == "coordinates":
                z = np.asarray(geo[k])
                f = z.flatten()
                geo[k] = np.dstack((f[1::2], f[::2])).reshape(z.shape).tolist()
            else:
                flip_geojson_coordinates(v)
    elif isinstance(geo, list):
        for k in geo:
            flip_geojson_coordinates(k)

In [5]:
flip_geojson_coordinates(nodes)
flip_geojson_coordinates(ways)

“Normalize” semi-structured JSON data into a flat table - much easier to work with! http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.io.json.json_normalize.html

In [6]:
import pandas as pd
nodes = pd.io.json.json_normalize(nodes['features'])
ways = pd.io.json.json_normalize(ways['features'])

Take a look at how many rows

In [23]:
len(nodes['id'])

192

In [10]:
len(ways['id'])

4719

Take a look at the gps coordinates

In [13]:
nodes['geometry.coordinates'][0:5]

0    [51.0745439, -115.4022874]
1    [43.2439679, -116.3783661]
2      [48.1056465, -1.9880204]
3      [49.6197688, 10.8350521]
4    [43.2669091, -122.4098458]
Name: geometry.coordinates, dtype: object

In [14]:
ways['geometry.coordinates'][0:2]

0    [[51.6368677, 7.624084], [51.6368396, 7.624103...
1    [[52.5380095, 13.3492891], [52.5380757, 13.349...
Name: geometry.coordinates, dtype: object

Fyi coordinates for ways is a linestring and will need only one pair for reverse geolocator. Later on I just extracted the first pair to map them on Carto DB along with the nodes points.

Look at powerplant names:

In [15]:
nodes['properties.name'].head()

0                                NaN
1                                NaN
2    Eolienne de la Ville es Archers
3    Biomasse Heizwerk Gerbersleithe
4     Clearwater Powerplant Number 2
Name: properties.name, dtype: object

In [16]:
nodes['properties.addr:country'].unique()
#will take more work to look at plants by country

array([nan, 'DE'], dtype=object)

In [18]:
#look into missingness in the data table - could be useful later on
nodes.isnull().sum()

geometry.coordinates                         0
geometry.type                                0
id                                           0
properties.abandoned                       191
properties.addr:city                       181
properties.addr:country                    191
properties.addr:postcode                   191
properties.addr:state                      185
properties.addr:street                     191
properties.branch                          191
properties.building                        189
properties.construction                    190
properties.description                     189
properties.designation                     189
properties.ele                             182
properties.fixme                           190
properties.frequency                       191
properties.fvst:navnelbnr                  191
properties.generator:method                157
properties.generator:output:electricity    187
properties.generator:output:heat           190
properties.ge

In [19]:
#for example, there are only 9 columns with less than 150 missing rows
df1 = nodes
df1 = (nodes.isnull().sum()) < 150
df1 = df1[df1 == True]
len(df1)

9

Look at fuel types

In [25]:
nodes['properties.generator:source'].unique()

array(['hydro', nan, 'wind', 'biomass', 'geothermal', 'nuclear', 'coal',
       'fossil', 'oil', 'solar', 'biogas'], dtype=object)

In [28]:
nodes['properties.plant:source'].unique()

array([nan, 'nuclear', 'hydro', 'biofuel', 'solar', 'biogas'], dtype=object)

In [27]:
ways['properties.generator:source'].unique()

array(['coal', nan, 'gas', 'hydro', 'fossil', 'oil', 'nuclear',
       'coal;oil;waste', 'waste', 'coal;gas', 'biofuel', 'diesel', 'solar',
       'biogas', 'gas;oil', 'diesel;gas', 'lignite', 'geothermal',
       'gas;diesel', 'biomass', 'coal;diesel', 'gas; coal; oil',
       'gas;coal', 'coal;oil', 'coal;gas;waste', 'wind', 'photovoltaic',
       'oil;gas', 'gas;oil;coal', 'coal;gas;oil', 'co-gen',
       'Gas; oil, diesel', 'oil/gas', 'Gas_or_Diesel', 'photovoltaik',
       'waste;gas'], dtype=object)

In [29]:
ways['properties.plant:source'].unique()

array([nan, 'nuclear', 'gas', 'solar', 'coal', 'hydro', 'coal;gas',
       'coal;gas;biomass', 'biomass', 'gas;steam', 'gas;oil', 'oil',
       'wind', 'coal;biomass', 'coal;biomass;waste', 'geothermal',
       'oil;gas', 'coal;oil', 'gas;coal', 'gas;coal;oil', 'waste',
       'waste;gas', 'biogas', 'wind;solar', 'Gas', 'diesel', 'osmosis',
       'peat;biomass', 'water', 'distillate oil', 'wood', ' solar',
       'biofuel', 'gas;solar'], dtype=object)

Export the tables to csv

In [ ]:
nodes.to_csv('osm_nodes_flip.csv')
ways.to_csv('osm_ways_flip.csv')

View the map!  Yellow dots are ways and red are nodes. 

https://jdills26.cartodb.com/viz/1bc14754-3c7c-11e6-88e8-0e98b61680bf/public_map

In [22]:
from IPython.display import HTML
HTML('<iframe width=100% height=520 frameborder=0 src=https://jdills26.cartodb.com/viz/1bc14754-3c7c-11e6-88e8-0e98b61680bf/embed_map allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>')

# FIN